In [11]:

import os 
import sys
sys.path.append(os.path.join(os.getcwd(),"..","Algorithm encapsulation"))
import pandas as pd
from Simulation import Simulation
import numpy as np

In [12]:
df=pd.read_csv("./a.csv",encoding="utf-8-sig",index_col=0,header=0)
phase = np.loadtxt("./data/phase")
ex=np.array([1,0,1,0,1,0,1,1,1,1,1,0])
vissim=Simulation(os.path.join(os.getcwd(),"..","vissim","aa","zhongsan.inp"),
               os.path.join(os.getcwd(),"..","vissim","aa","vissim.ini"),
               os.path.join(os.getcwd(),"..","vissim","aa","zhongsan - 副本.inp"))
vissim.set_dwelltime(np.array([df.dw1,df.dw2]))
vissim.init_vissim()
scs=vissim.scs

for sc in scs:
    id=int(sc.ID)
    vissim.set_offset(sc,df.offset[id-1])
    vissim.set_cycle(sc,df.z[id-1])
    sgs=sc.SignalGroups
    g=phase[id-1]*df.z[id-1]
    vissim.Set_SignalGroups(sgs,g)
df["d"]=df.distance
car_t=np.array([df.d/df.t1,df.d/df.t2])
bus_t=np.array([df.d/(df.tb1-ex*df.dw1[:12]),df.d/(df.tb2-ex*df.dw2[:12])])
vissim.set_desiredspeed(car_t,bus_t)
vissim.set_vehInput(1)
vissim.set_period(3600)
vissim.start()

In [14]:
tz= pd.read_table(os.path.join(os.getcwd(),"..","vissim","aa","zhongsan - 副本.fzp"), encoding='gbk')

In [15]:
index=np.where(tz[tz.columns[0]].str.startswith(' Link;'))[0]
data=tz[index[0]:]
data.columns = ['data']
data = pd.DataFrame([var.split(';') for var in data['data']])
data.columns = data.iloc[0, :].apply(lambda x :x.strip())
data = data.drop([0], axis=0)
data=data.drop('',axis=1)
data=data.reset_index(drop=True)
t=data[data.columns].astype(float)
data[t.columns]=t

In [16]:
links=vissim.vnet.Links
linkName=dict()
for link in links:
    linkName[link.ID]=link.Name
data["LinkName"]=data["Link"].apply(lambda x:linkName[int(x)])
data=data[-(data["LinkName"]=="")]

In [21]:
data["speed"]=data["v"]//10*10

In [55]:
dgp=data.groupby(["speed","LinkName"])["TQDelay"].aggregate(np.mean)

In [79]:
traveltime = pd.read_table(os.path.join(os.getcwd(),"..","vissim","aa",'zhongsan - 副本.rsz'), encoding='gbk')
traveltime = traveltime[9:].reset_index(drop=True)
traveltime.columns = ['data']
traveltime1 = pd.DataFrame([var.split(';') for var in traveltime.data])
traveltime1 = traveltime1.drop(columns=[len(traveltime1.columns)-1], axis=1)[1:]
traveltime1.columns = ["time"]+["traveltime", "vehicles"]*8
traveltime1=traveltime1[3:].reset_index(drop=True)
traveltime1.replace('\s+','',regex=True,inplace=True)
traveltime1.columns=traveltime1.columns+traveltime1.loc[0,:]
traveltime1=traveltime1[2:].reset_index(drop=True)



In [98]:
delay = pd.read_table(os.path.join(os.getcwd(),"..","vissim","aa",'zhongsan - 副本.vlz'), encoding='gbk')
delay = delay[9:].reset_index(drop=True)
delay.columns = ['data']
delay1 = pd.DataFrame([var.split(';') for var in delay.data])
delay1 = delay1.drop(columns=[len(delay1.columns)-1], axis=1)[1:]
delay1.columns = ["time"]+["delay", "stoped", "stops", "vehicle", "per delay", "pers"]*8
delay1.replace('\s+','',regex=True,inplace=True)
delay1.columns=delay1.columns+delay1.loc[4,:]
delay1=delay1.drop(11,axis=0)
delay1=delay1[4:].reset_index(drop=True)
